In [0]:
%python
%pip install mlflow==1.14.0
import boto3
import pandas as pd

In [0]:
# Set up S3 client and bucket
s3 = boto3.client('s3')
bucket = "columbia-gr5069-main"

# List of files you want to load from the bucket
keys = {
    "drivers": "raw/drivers.csv",
    "races": "raw/races.csv",
    "results": "raw/results.csv",
    "constructors": "raw/constructors.csv",
}

# Dictionary to store the loaded DataFrames
dataframes = {}

# Loop through and load each CSV into a DataFrame
for name, key in keys.items():
    obj = s3.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(obj['Body'])
    dataframes[name] = df
    print(f"Loaded {name} ({df.shape[0]} rows, {df.shape[1]} columns)")

# Example usage:
drivers_df = dataframes['drivers']
races_df = dataframes['races']
results_df = dataframes['results']
constructors_df = dataframes['constructors']

# Preview a DataFrame
display(drivers_df)
display(races_df)
display(results_df)
display(constructors_df)